In [2]:
import numpy as np
from tensorflow import keras
from keras.constraints import maxnorm
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical   


(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
print("Shape of training data:")
print(X_train.shape)
print(y_train.shape)
print("Shape of test data:")
print(X_test.shape)
print(y_test.shape)

Shape of training data:
(50000, 32, 32, 3)
(50000, 1)
Shape of test data:
(10000, 32, 32, 3)
(10000, 1)


In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
class_num = y_test.shape
#print(X_train[0])
print(y_train[0])

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [5]:
X_train = keras.applications.inception_v3.preprocess_input(X_train)
X_test = keras.applications.inception_v3.preprocess_input(X_test)

In Keras, the Lambda layer allows you to wrap an arbitrary function or lambda expression and use it as a layer in a neural network model. The Lambda layer takes a function or lambda expression as its first argument, and applies that function to each element of the input tensor.

//3 channels of arbitrary shape
inp = Input(shape=(None, None, 3))

Inception Layer starts with 229X229X3

set base_model, trainable = False, this means that the layers won't learn and we are just going to use what they learnt from imagenet

In [6]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D,Dropout
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam
from keras.layers import Input
from tensorflow.keras.layers import Lambda

base_inception = InceptionV3(include_top=False,
                             input_shape=(299, 299, 3))

inp_layer = Input(shape=(32, 32, 3))

resizing_layer = Lambda(lambda image:keras.preprocessing.image.smart_resize(image, (299, 299)))(inp_layer)

x = base_inception(resizing_layer, training=False)

#x = base_inception.output
x= GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
output_layer =  Dense(10, activation='softmax')(x)

model = Model(inputs=inp_layer, outputs=output_layer)
     

In [ ]:
base_inception.trainable = False


model.compile(keras.optimizers.Adam(learning_rate=.001),loss='categorical_crossentropy',metrics=['accuracy'])

h1 = model.fit(X_train,y_train,validation_data=(X_test, y_test),batch_size=256,epochs=10,verbose=1)


Epoch 1/10
196/196 [==============================] - 2644s 13s/step - loss: 2.2619 - accuracy: 0.1355 - val_loss: 2.1660 - val_accuracy: 0.1650
Epoch 2/10
196/196 [==============================] - 2675s 14s/step - loss: 2.0419 - accuracy: 0.2179 - val_loss: 1.8923 - val_accuracy: 0.2712
Epoch 3/10
196/196 [==============================] - 2688s 14s/step - loss: 1.9105 - accuracy: 0.2636 - val_loss: 1.8634 - val_accuracy: 0.3075
Epoch 4/10
196/196 [==============================] - 2925s 15s/step - loss: 1.8564 - accuracy: 0.2869 - val_loss: 1.7999 - val_accuracy: 0.3208
Epoch 5/10
196/196 [==============================] - 3015s 15s/step - loss: 1.8239 - accuracy: 0.3038 - val_loss: 1.8307 - val_accuracy: 0.3190
Epoch 6/10
196/196 [==============================] - 2853s 15s/step - loss: 1.8063 - accuracy: 0.3118 - val_loss: 1.7207 - val_accuracy: 0.3564
Epoch 7/10
196/196 [==============================] - 2725s 14s/step - loss: 1.8097 - accuracy: 0.3132 - val_loss: 1.7691 - val_ac

In [ ]:
results = model.evaluate(X_test_p, Y_test_p)

Fine tuning

After training the model this far, unfreeze some layers in the base_inception model (our pre-trained model from keras applications). Then we will jointly train both these layers and the part that we added (to the base_inception model)
As a starting point I have chosen to unfreeze the layers from the 164th layer(exactly starting from the mixed5 layer in the inception model!), then I will recompile the model(make sure that the model wont overfit)

In [ ]:

for layer in base_inception.layers[:164]:
  layer.trainable=False

for layer in base_inception.layers[164:]:
  layer.trainable=True

my_opt = K.optimizers.Adam(1e-5)
model.compile(optimizer=my_opt,loss='categorical_crossentropy',metrics=['accuracy'])

h2 = model.fit(X_train_p,Y_train_p,validation_data=(X_test_p, Y_test_p), batch_size=300,epochs=4,verbose=1)

results = model.evaluate(X_test_p, Y_test_p)